## ROC extraction

15/06/16 by greghor


on commence par regarder les documents ou l on observe le mot center
utilise les fichiers .html
integre corrections typographiques pour coller à la solution
integre ROC à partir des currencies

In [1]:
# SYSTEM
from os import listdir
import sys
sys.path.insert(0,'/Users/greghor/Documents/taf/UGent/python_prog/py_commonFunc/')
from PyCommonFun import *

#NLTK
import nltk, re, pprint
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import words
from nltk.tokenize import RegexpTokenizer
from nltk.metrics import *
from nltk.text import Text

#GENERAL
import numpy as np
import pandas as pd
import pickle
import string

In [2]:
# data path
curDir=%pwd
if curDir=='/Users/greghor/Documents/taf/dataScience/Euroclear_hackathon/notebook':
    dataPath='/Users/greghor/Documents/taf/dataScience/Euroclear_hackathon/dataset/data_0_labels' 
                   
else:
    dataPath='../../data_0'

In [3]:
# fonctions used in the notebook

def getContext(s,pattern,context=list(),width=100):
    
    
    loc=s.find(pattern)
    
    if loc == -1:
        return context
    elif loc-width < 0:
        context.append(s[:loc+width])
        return getContext(s[loc+len(pattern):],pattern,context,width)
    elif loc + width > len(s):
        context.append(s[loc-width:])
        return getContext(s[loc+len(pattern):],pattern,context,width)
    else:
        context.append(s[loc-width:loc+width])
        return getContext(s[loc+len(pattern):],pattern,context,width)
    
    
def useSolutionTypo(s):
    s=s.lower()
    if s=="sao paulo":
        s="sao-paulo"
    elif s=='target 1' or s =='target 2' or s == 'target' or s=='target1':
        s='target holidays'
    elif s=='euroclear':
        s=''
    return s



def document_from_hmtl(doc_path):
    with open(doc_path) as html_file:
        full_text = html_file.read()
        body = re.findall(r'<body[^>]*?>(.*?)</body>', full_text)
        cleantext = re.sub('<\/?span[^>]*>', ' ', body[0])
        clean_text = re.sub('<.*?>', ' ', cleantext)
        clean_text = clean_text.lower()
        clean_text = re.sub('0.01', '1', clean_text)
        clean_text = re.sub(',', '', clean_text)
        clean_text = re.sub(r'[^a-z0-9]', ' ', clean_text)
        clean_text = re.sub('\s+', ' ', clean_text)
        return clean_text


In [4]:
htmlFiles=listdir('../../data_0/html/')
PyFind(htmlFiles, lambda x: '' )

[]

In [5]:
# get the solution
labels=pd.read_csv(dataPath + '/labels.csv')
ROCTmp=pd.read_csv('ROC_fullList.csv',delimiter=";")
cities_groundTruth=pd.read_csv(dataPath + '/cities.csv')
ROC_list=['target 1','target 2','target'] # add target 1 and target 2
# lowerise ROC and remove ponctuation
for roc in ROCTmp['CIT_NM'].values:
    roc=str(roc).lower().translate(None, string.punctuation)
    ROC_list.append(roc.rstrip())

In [6]:
# add roc from operationnal curency
roc_fromCurrency=pd.read_csv('currency_ROC_fullList.csv',delimiter=";")
# merge with labels on operationnal currency
roc_fromCurrency=pd.merge(left=roc_fromCurrency,right=labels,left_on='Currency Code', right_on='OpCurrency')
# keep only currency code and default ROC columns
col=["isin","OpCurrency","Default ROC",]
roc_fromCurrency=roc_fromCurrency[col]
roc_fromCurrency.rename(columns={'Default ROC':'currency_ROC'},inplace=True)
roc_fromCurrency.head()

,isin,OpCurrency,currency_ROC
0,XS1293836406,AUD,Sydney|Melbourne
1,XS1293840770,AUD,Sydney|Melbourne
2,XS1293141849,AUD,Sydney|Melbourne
3,XS1340008645,AUD,Sydney|Melbourne
4,XS1267292354,AUD,Sydney|Melbourne


In [7]:
gf=0


context_pat=dict()
htmlFilesList=listdir('../../data_0/html/')

for isin in np.unique(labels['isin'].values):
    
    fid=labels['fileId'].loc[labels['isin']==isin]
    
    text=''
    for f in fid:

        indTmp=PyFind(htmlFiles, lambda x: f in x )
        fidPath=htmlFilesList[indTmp[0]]
        text=document_from_hmtl(dataPath + '/html/' + fidPath)

        context_pat[isin]=dict()
        sentence=sent_tokenize(text)

        pattern=["Business Day Centre","Financial Centre","Business Centre","Currency Business Day",'Business Day']
    
    for pat in pattern:
        context_pat[isin][pat]=getContext(text,pat.lower(),[],150)
        # il faut impérativement passer, un troisieme argument vide pour que le code 
        # fonctionne mais je ne comprends pas bien pourquoi
            
            
            
    gf+=1
       


In [8]:
# try to extract the ROC 

extractedROC_Raw_dict=dict() # contains only cities extracted from the documents 
extractedROC_Curr_dict=dict() # add the default ROC from currency 


for isin in context_pat.keys():
    
    extractedROC_Raw_dict[isin]=dict()
    extractedROC_Curr_dict[isin]=dict()
    
    currencyROCTmp=roc_fromCurrency['currency_ROC'].loc[roc_fromCurrency['isin']==isin].values[0].lower()
    currencyROCTmp=useSolutionTypo(currencyROCTmp)
    
    extractedROC_Raw_dict[isin]['currencyROC']=currencyROCTmp
    extractedROC_Curr_dict[isin]['currencyROC']=currencyROCTmp
    
    for pat in context_pat[isin].keys():
    
        extractedROC_Raw_dict[isin][pat]=''
        extractedROC_Curr_dict[isin][pat]=currencyROCTmp
        
        for roc in ROC_list:
            
            if roc in ''.join(context_pat[isin][pat]):
                if len(extractedROC_Raw_dict[isin][pat])==0:
                    extractedROC_Raw_dict[isin][pat]=useSolutionTypo(roc)
                else:
                    extractedROC_Raw_dict[isin][pat]=extractedROC_Raw_dict[isin][pat]+'|'+ useSolutionTypo(roc)
                    
                if useSolutionTypo(roc) not in  extractedROC_Curr_dict[isin][pat]:
                    extractedROC_Curr_dict[isin][pat]=extractedROC_Curr_dict[isin][pat]+'|'+ useSolutionTypo(roc)


                    

                
                 

## changer la ligne extractedROC_dict=extractedROC_Curr_dict



In [9]:

extractedROC_dict=extractedROC_Raw_dict
extractedROC_df=pd.DataFrame()
extractedROC_df['isin']=[]
for pat in extractedROC_dict['XS1289653195'].keys():
    extractedROC_df[pat]=[]




In [10]:
for isin in extractedROC_dict.keys():
    newRow=[isin]
    for pat in extractedROC_dict[isin].keys():
        newRow.append(extractedROC_dict[isin][pat].upper())
    extractedROC_df.loc[len(extractedROC_df)]=newRow

In [11]:
print isin
print pat
print extractedROC_dict[isin][pat]

XS1349021326
Business Centre



In [12]:
extractedROC_dict['XS1240015799']['Business Centre']

''

In [13]:
'new york' not in extractedROC_dict['XS1240015799']['Business Centre']

True

In [17]:
extractedROC_df

,isin,Financial Centre,currencyROC,Currency Business Day,Business Day Centre,Business Day,Business Centre
0,XS1240015799,NEW YORK,NEW YORK,NEW YORK,,NEW YORK,
1,XS1293137904,,TARGET HOLIDAYS,,TARGET HOLIDAYS,TARGET HOLIDAYS,
2,XS1313796846,LONDON|TOKYO,TOKYO,,,,
3,XS1051380837,,TOKYO,,,,
4,XS1313800895,LONDON|TOKYO,TOKYO,,,,OSAKA
5,XS1267246327,TARGET HOLIDAYS|NEW YORK,NEW YORK,,,TARGET HOLIDAYS,TARGET HOLIDAYS
6,XS1275964507,LONDON|NEW YORK|SAO-PAULO|TOKYO,TOKYO,,,LONDON|NEW YORK|SAO-PAULO|TOKYO,
7,XS1329979261,TOKYO,TOKYO,,,,
8,XS1326567895,HONG KONG,HONG KONG,,,,
9,XS1253932765,NEW YORK|SINGAPORE,NEW YORK,,,BRUSSELS||LUXEMBOURG|NEW YORK|SINGAPORE,


In [21]:
# rank the priority of the pattern
patternRank_list=[1,0,2,3,4]
patternRank_dict=dict()
for gi in range(len(patternRank_list)):
    patternRank_dict[pattern[gi]]=patternRank_list[gi]
patternRank_dict

{'Business Centre': 2,
 'Business Day': 4,
 'Business Day Centre': 1,
 'Currency Business Day': 3,
 'Financial Centre': 0}

In [42]:
extractedROC_singleSol=pd.DataFrame()
col=['isin','solCities','pattern']
for col in col:
    extractedROC_singleSol[col]=''
#extractedROC_df[['isin','ext_Cities','pattern']]
gf=0;
for isin in extractedROC_df['isin'].values:
    #print isin
    newRow=[isin]
    gpat=0
    while gpat <= 3 and len(extractedROC_df[pattern[gpat]].values[gf])==0:
        gpat+=1
    newRow.append(extractedROC_df[pattern[gpat]].values[gf])
    newRow.append(pattern[gpat])
    extractedROC_singleSol.loc[len(extractedROC_singleSol)]=newRow
    gf+=1

In [43]:
# add the default currency 
extractedROC_singleSol=pd.merge(extractedROC_singleSol,extractedROC_df[['currencyROC','isin']],on='isin')

In [44]:
extractedROC_singleSol.head()

,isin,solCities,pattern,currencyROC
0,XS1240015799,NEW YORK,Financial Centre,NEW YORK
1,XS1293137904,TARGET HOLIDAYS,Business Day Centre,TARGET HOLIDAYS
2,XS1313796846,LONDON|TOKYO,Financial Centre,TOKYO
3,XS1051380837,,Business Day,TOKYO
4,XS1313800895,LONDON|TOKYO,Financial Centre,TOKYO


In [45]:
# merge currencyROC and extracted ROC
gl=0
for val in extractedROC_singleSol['currencyROC'].values:
    for curr in val.split('|'):
        if len(extractedROC_singleSol['solCities'].values[gl])==0:
            extractedROC_singleSol['solCities'].values[gl]=curr
        if curr not in extractedROC_singleSol['solCities'].values[gl]:
            extractedROC_singleSol['solCities'].values[gl]=extractedROC_singleSol['solCities'].values[gl] + '|' + curr
    gl+=1


In [46]:
extractedROC_singleSol

,isin,solCities,pattern,currencyROC
0,XS1240015799,NEW YORK,Financial Centre,NEW YORK
1,XS1293137904,TARGET HOLIDAYS,Business Day Centre,TARGET HOLIDAYS
2,XS1313796846,LONDON|TOKYO,Financial Centre,TOKYO
3,XS1051380837,TOKYO,Business Day,TOKYO
4,XS1313800895,LONDON|TOKYO,Financial Centre,TOKYO
5,XS1267246327,TARGET HOLIDAYS|NEW YORK,Financial Centre,NEW YORK
6,XS1275964507,LONDON|NEW YORK|SAO-PAULO|TOKYO,Financial Centre,TOKYO
7,XS1329979261,TOKYO,Financial Centre,TOKYO
8,XS1326567895,HONG KONG,Financial Centre,HONG KONG
9,XS1253932765,NEW YORK|SINGAPORE,Financial Centre,NEW YORK


In [48]:
# merge with citiesGreg to check how many results are OK

citiesGreg=pd.read_csv('citiesGreg.csv')
joinedCities=pd.merge(citiesGreg,extractedROC_singleSol,on='isin')

In [49]:
joinedCities

,Unnamed: 0,isin,citiesName,fileId,solCities,pattern,currencyROC
0,0,XS1240015799,NEW YORK,0900045c83a87e4a,NEW YORK,Financial Centre,NEW YORK
1,1,XS1293137904,TARGET HOLIDAYS,0900045c83c101a8,TARGET HOLIDAYS,Business Day Centre,TARGET HOLIDAYS
2,2,XS1313796846,LONDON|TOKYO,0900045c83ac888d,LONDON|TOKYO,Financial Centre,TOKYO
3,3,XS1313802677,NEW YORK|TOKYO,0900045c83b42aaf,NEW YORK|TOKYO,Financial Centre,NEW YORK
4,4,XS1313800895,LONDON|TOKYO,0900045c83b36448,LONDON|TOKYO,Financial Centre,TOKYO
5,5,XS1267246327,NEW YORK|TARGET HOLIDAYS,0900045c83a715b0,TARGET HOLIDAYS|NEW YORK,Financial Centre,NEW YORK
6,6,XS1275964507,LONDON|NEW YORK|SAO-PAULO|TOKYO,0900045c8365c27b,LONDON|NEW YORK|SAO-PAULO|TOKYO,Financial Centre,TOKYO
7,7,XS1329979261,TOKYO,0900045c83aab72e,TOKYO,Financial Centre,TOKYO
8,8,XS1312160507,NEW YORK,0900045c83af0dc8,NEW YORK,Financial Centre,NEW YORK
9,9,XS1253932765,NEW YORK|SINGAPORE,0900045c833b5bd8,NEW YORK|SINGAPORE,Financial Centre,NEW YORK


In [112]:
# check the compliance between ground truth and my solution
for pat in extractedROC_dict['XS1289653195'].keys():
    indCheck=[]
    gval=0
    for multiCity in joinedCities[pat].values:
        indCheckTmp=True
        for city in multiCity.split('|'): # check that all extracted cities are in the ground Truth
            indCheckTmp=indCheckTmp and (city in  joinedCities['citiesName'].values[gval])
        
        # check that ground truth do not content additionnal city
        if len(joinedCities['citiesName'].values[gval]) == len(multiCity):
            indCheckTmp=indCheckTmp and True
        else:
            indCheckTmp=indCheckTmp and False
        indCheck.append(indCheckTmp)
        gval+=1
    joinedCities["check_" + pat ]=indCheck

# reorder the columns for clarity
col=['fileId',
 'isin',
 'citiesName',
 'Financial Centre','check_Financial Centre',
 'Business Day Centre', 'check_Business Day Centre',
 'Business Centre','check_Business Centre',
 'Currency Business Day', 'check_Currency Business Day',
 'Business Day', 'check_Business Day',
  'currencyROC','check_currencyROC']


In [113]:
extractedROC_dict['XS1275964507']['Financial Centre'].split('|')

['tokyo', 'london', 'new york', 'sao-paulo']

In [114]:
check_extractedCities=joinedCities[col]
check_extractedCities.to_csv('check_extractedCities.csv')

In [115]:
check_extractedCities

,fileId,isin,citiesName,Financial Centre,check_Financial Centre,Business Day Centre,check_Business Day Centre,Business Centre,check_Business Centre,Currency Business Day,check_Currency Business Day,Business Day,check_Business Day,currencyROC,check_currencyROC
0,0900045c83a87e4a,XS1240015799,NEW YORK,NEW YORK,True,NEW YORK,True,NEW YORK,True,NEW YORK,True,NEW YORK,True,NEW YORK,True
1,0900045c83c101a8,XS1293137904,TARGET HOLIDAYS,TARGET HOLIDAYS,True,TARGET HOLIDAYS,True,TARGET HOLIDAYS,True,TARGET HOLIDAYS,True,TARGET HOLIDAYS,True,TARGET HOLIDAYS,True
2,0900045c83ac888d,XS1313796846,LONDON|TOKYO,TOKYO|LONDON,True,TOKYO,False,TOKYO,False,TOKYO,False,TOKYO,False,TOKYO,False
3,0900045c83b42aaf,XS1313802677,NEW YORK|TOKYO,NEW YORK|TOKYO,True,NEW YORK,False,NEW YORK|OSAKA,False,NEW YORK,False,NEW YORK,False,NEW YORK,False
4,0900045c83b36448,XS1313800895,LONDON|TOKYO,TOKYO|LONDON,True,TOKYO,False,TOKYO|OSAKA,False,TOKYO,False,TOKYO,False,TOKYO,False
5,0900045c83a715b0,XS1267246327,NEW YORK|TARGET HOLIDAYS,NEW YORK|TARGET HOLIDAYS,True,NEW YORK,False,NEW YORK|TARGET HOLIDAYS,True,NEW YORK,False,NEW YORK|TARGET HOLIDAYS,True,NEW YORK,False
6,0900045c8365c27b,XS1275964507,LONDON|NEW YORK|SAO-PAULO|TOKYO,TOKYO|LONDON|NEW YORK|SAO-PAULO,True,TOKYO,False,TOKYO,False,TOKYO,False,TOKYO|LONDON|NEW YORK|SAO-PAULO,True,TOKYO,False
7,0900045c83aab72e,XS1329979261,TOKYO,TOKYO,True,TOKYO,True,TOKYO,True,TOKYO,True,TOKYO,True,TOKYO,True
8,0900045c83af0dc8,XS1312160507,NEW YORK,NEW YORK,True,NEW YORK,True,NEW YORK,True,NEW YORK,True,NEW YORK,True,NEW YORK,True
9,0900045c833b5bd8,XS1253932765,NEW YORK|SINGAPORE,NEW YORK|SINGAPORE,True,NEW YORK,False,NEW YORK,False,NEW YORK,False,NEW YORK|BRUSSELS|LUXEMBOURG|SINGAPORE,False,NEW YORK,False


In [117]:
len(PyFind(check_extractedCities['check_Financial Centre'].values, lambda x: x==True))/np.float(len(check_extractedCities['check_Financial Centre'].values))

0.7881567973311092

In [62]:
check_extractedCities['check_Business Day'].values or check_extractedCities['check_Financial Centre'].values

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [72]:
checkAll=np.logical_or(check_extractedCities['check_Financial Centre'].values, check_extractedCities['check_Business Day Centre'].values)
checkAll=np.logical_or(check_extractedCities['check_Business Day'].values, checkAll)
checkAll=np.logical_or(check_extractedCities['check_Business Day'].values, checkAll)



In [73]:
len(checkAll==True)

1199

In [ ]:
col=[col for col in check_extractedCities.columns if col.startswith('check')]
print col

for gi in range(len(col)-1):
    checkAll=np.logical_or(check_extractedCities[col[gi]].values,check_extractedCities[col[gi+1]].values)

checkAll.shape

In [ ]:
len(PyFind(checkAll,lambda x: x==True))/1199.

In [ ]:
check_extractedCities.head()

In [ ]:
np.where(text=='Issuer')

In [ ]:
toto=re.search('Business Day Centre',text)

In [ ]:
toto.group()

In [ ]:
toto=text.common_contexts("Centre")

In [ ]:
text.findall("<.*><Day Centre><.*>")

In [ ]:
moby = Text(nltk.corpus.gutenberg.words('melville-moby_dick.txt'))

In [ ]:
moby

In [ ]:
moby.common_contexts('whale')

In [ ]:
toto

In [ ]:
toto

In [ ]:
text.concordance('prospectus')


In [ ]:
type(text)

In [ ]:
f

In [ ]:
f

In [ ]:
f

In [ ]:
for f in labels[0

In [ ]:
labels.fileId.values[0]

In [ ]:
for f in labels.fileId:
    print f